In [45]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
from skimage import feature
from skimage import io
from skimage import color, morphology
from pystackreg import StackReg
import gc
from pystackreg.util import to_uint16
%matplotlib inline

In [46]:
#specify import files
#import image and put into list (firstlist)
input_dir="C:\\Users\\sunadome\\Desktop\\cantas_analysis\\exp211216_axioloid_hybiss_reanalysis"
tissue_num="tissue3"

os.mkdir(f"E:\\HybISS_output\\exp211216_cantas_reanalysis\\output\\{tissue_num}")#Make output folder
os.mkdir(f"E:\\HybISS_output\\exp211216_cantas_reanalysis\\output\\{tissue_num}\\image_results")#This folder will have all the drawn images: coordinates drawn on img_written
output_dir=f"E:\\HybISS_output\\exp211216_cantas_reanalysis\\output\\{tissue_num}"

genedecode_filename="hox_decode_exp211101.csv" #This is a file that contain "Gene" name and "Decode" number
img_written=f"{tissue_num}_round1_c2.png"   #This is image in which the coordinates of amplicons will be drawn
df_genedecode=pd.read_csv(f"{input_dir}\\{genedecode_filename}")#Make dataframe for gene name and corresponding decoding list



firstlist=[]

r1_seq1=cv2.imread(f"{input_dir}\\input_image\\{tissue_num}\\round1_c5.tif", cv2.IMREAD_GRAYSCALE)
firstlist.append(r1_seq1)
r1_seq2=cv2.imread(f"{input_dir}\\input_image\\{tissue_num}\\round1_c3.tif", cv2.IMREAD_GRAYSCALE)
firstlist.append(r1_seq2)
r1_seq3=cv2.imread(f"{input_dir}\\input_image\\{tissue_num}\\round1_c1.tif", cv2.IMREAD_GRAYSCALE)
firstlist.append(r1_seq3)
r1_seq4=cv2.imread(f"{input_dir}\\input_image\\{tissue_num}\\round1_c4.tif", cv2.IMREAD_GRAYSCALE)
firstlist.append(r1_seq4)


r2_seq1=cv2.imread(f"{input_dir}\\input_image\\{tissue_num}\\round2_c5.tif", cv2.IMREAD_GRAYSCALE)
firstlist.append(r2_seq1)
r2_seq2=cv2.imread(f"{input_dir}\\input_image\\{tissue_num}\\round2_c3.tif", cv2.IMREAD_GRAYSCALE)
firstlist.append(r2_seq2)
r2_seq3=cv2.imread(f"{input_dir}\\input_image\\{tissue_num}\\round2_c1.tif", cv2.IMREAD_GRAYSCALE)
firstlist.append(r2_seq3)
r2_seq4=cv2.imread(f"{input_dir}\\input_image\\{tissue_num}\\round2_c4.tif", cv2.IMREAD_GRAYSCALE)
firstlist.append(r2_seq4)


r3_seq1=cv2.imread(f"{input_dir}\\input_image\\{tissue_num}\\round3_c5.tif", cv2.IMREAD_GRAYSCALE)
firstlist.append(r3_seq1)
r3_seq2=cv2.imread(f"{input_dir}\\input_image\\{tissue_num}\\round3_c3.tif", cv2.IMREAD_GRAYSCALE)
firstlist.append(r3_seq2)
r3_seq3=cv2.imread(f"{input_dir}\\input_image\\{tissue_num}\\round3_c1.tif", cv2.IMREAD_GRAYSCALE)
firstlist.append(r3_seq3)
r3_seq4=cv2.imread(f"{input_dir}\\input_image\\{tissue_num}\\round3_c4.tif", cv2.IMREAD_GRAYSCALE)
firstlist.append(r3_seq4)


r4_seq1=cv2.imread(f"{input_dir}\\input_image\\{tissue_num}\\round4_c5.tif", cv2.IMREAD_GRAYSCALE)
firstlist.append(r4_seq1)
r4_seq2=cv2.imread(f"{input_dir}\\input_image\\{tissue_num}\\round4_c3.tif", cv2.IMREAD_GRAYSCALE)
firstlist.append(r4_seq2)
r4_seq3=cv2.imread(f"{input_dir}\\input_image\\{tissue_num}\\round4_c1.tif", cv2.IMREAD_GRAYSCALE)
firstlist.append(r4_seq3)
r4_seq4=cv2.imread(f"{input_dir}\\input_image\\{tissue_num}\\round4_c4.tif", cv2.IMREAD_GRAYSCALE)
firstlist.append(r4_seq4)



#specify the number of round and color(used in in situ seq)
num_rnd=4
num_clr=4


In [47]:
# to find which row/column number can be used to divide the image
"""
def prime_factorize(n):
    a = []
    while n % 2 == 0:
        a.append(2)
        n //= 2
    f = 3
    while f * f <= n:
        if n % f == 0:
            a.append(f)
            n //= f
        else:
            f += 2
    if n != 1:
        a.append(n)
    return a

import collections

pxl_y=3782
pxl_x=3789
y_divide=collections.Counter(prime_factorize(pxl_y))
x_divide=collections.Counter(prime_factorize(pxl_x))

print(y_divide)
print(x_divide)
"""

Counter({2: 1, 31: 1, 61: 1})
Counter({3: 2, 421: 1})


In [48]:
#each image size after division, which can be decided based on "x_divide" and "y_divide" above
size=(1200,2000)
rows = int(np.ceil(r1_seq1.shape[0] / size[0]))  # the number of rows
cols = int(np.ceil(r1_seq1.shape[1] / size[1]))  # the number of columns


4
1
<class 'numpy.ndarray'>


In [49]:
#perform top hat transform to all of the images
#kernel:np.ones(n,n).....n/2 corresponds to radius of minimumaizing and maximizing of morphological opening in Fiji. Also, n/2 is similar value to the radius of amplicon's pxl size
#make list where processd images are contained:secondlist

kernel = np.ones((5, 5), np.uint8)
secondlist=[]

for i in range(len(firstlist)):
    src=firstlist[i]
    dst=cv2.morphologyEx(src, cv2.MORPH_TOPHAT, kernel)
    secondlist.append(dst)

del firstlist
#fisrstlist is not needed anymore

In [50]:
#produce maximum intensity projection from top hat-transfomed images for each round and put into list:thirdlist
thirdlist=[]

list=[secondlist[i:i+num_clr] for i in range(0,len(secondlist),num_clr)]

for j in range(num_rnd):
    array=np.array(list[j])
    mip=np.amax(array, axis=0)
    thirdlist.append(mip)

del list
 

In [51]:
#split images in secondlist(each chanel) and put them to list:splitraw
#split images in thirdlist(mip) and put them to list:splitmip
splitorg=[]
for i in range(len(secondlist)):
    for row_img in np.array_split(secondlist[i], rows, axis=0):
        for chunk in np.array_split(row_img, cols, axis=1):
            splitorg.append(chunk)

splitmip=[]
for i in range(len(thirdlist)):
    for row_img in np.array_split(thirdlist[i], rows, axis=0):
        for chunk in np.array_split(row_img, cols, axis=1):
            splitmip.append(chunk)



#align image 
sr = StackReg(StackReg.RIGID_BODY)

# register 2nd image to 1st
# i:for round
# j:for split image


mip_align=splitmip[0:rows*cols]
org_align=[splitorg[i*rows*cols:i*rows*cols+rows*cols] for i in range(num_clr)]
img_mipref=splitmip[0:rows*cols]

for i in range(1,num_rnd):
    org_list=splitorg[i*rows*cols*num_clr:i*rows*cols*num_clr+rows*cols*num_clr]
    mip_list=splitmip[i*rows*cols:i*rows*cols+rows*cols]
    
   
    listorg=[]
    
    for j in range(rows*cols):
        reference=np.array(img_mipref[j])
        register=np.array(mip_list[j])
        sr.register(reference, register)
        
        
        org_list_sub=org_list[j:len(org_list):rows*cols]
        
        for k in range(num_clr):
            frame=np.array(org_list_sub[k])
            frame_aligned=sr.transform(frame)
            frame_aligned_16bit=to_uint16(frame_aligned)
            frame_aligned_16bit_modified=np.where(frame_aligned_16bit>255, 255, frame_aligned_16bit)
            frame_aligned_8bit=frame_aligned_16bit_modified.astype(np.uint8)
            
            listorg.append(frame_aligned_8bit)
            
                        
    
    
    listsort=[listorg[m:len(listorg):num_clr] for m in range(num_clr)]
    
    org_align.extend(listsort)
        
  

del secondlist
del thirdlist
del splitorg
del splitmip
del listorg
del mip_align
del img_mipref
del listsort

gc.collect()

2584

In [52]:
#make round- and color- list for image name

round_list=[]
for i in range(num_clr*num_rnd):
    x=(i+num_clr)//num_clr
    round_list.append(x)

color_list=[]
for i in range(num_clr*num_rnd):
    x=i%num_clr+1
    color_list.append(x)

print(color_list)
print(round_list)



[1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4]
[1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4]


In [53]:
#concatenate and put aligned images into finallist as well as export

for i in range(len(org_align)):
    image=org_align[i]
    image_array=np.array(image)
    img_concat=cv2.hconcat(image_array[0:cols])
    for j in range(cols, len(image), cols):
        v=cv2.hconcat(image_array[j:j+cols])
        img_concat=cv2.vconcat([img_concat,v])
    org_align[i]=img_concat
    
    cv2.imwrite(f"{output_dir}\\round{round_list[i]}_seq{color_list[i]}.tif", img_concat)


finallist=org_align[:]
                      

del org_align

gc.collect()



34

In [54]:
#find amplicon in first image
first_rnd=np.array(finallist[0:num_clr])
ref_frame=np.amax(first_rnd, axis=0)

from skimage import feature
c=feature.blob_log(ref_frame, min_sigma=1, max_sigma=3, num_sigma=2,threshold=0.05, overlap=0.5, log_scale=False, exclude_border=False)


In [55]:
#change the dtype of coordinate of amplicon from float to int
y_float=c[:,0].tolist()
x_float=c[:,1].tolist()

x=[]
y=[]
for i in range(len(c[:])):
    a=int(x_float[i])
    x.append(a)
    b=int(y_float[i])
    y.append(b)

del y_float
del x_float


In [56]:
#create Dataframe for output
xy_pos={"x_pos" : x, "y_pos" : y}
df = pd.DataFrame.from_dict(xy_pos, orient='index').T

#retrieve intensity at each coordinate
decode=np.zeros((len(c[:,]), num_rnd), dtype=np.uint8)

for i in range(0, num_rnd*num_clr, num_clr):
    
    brightarray=np.zeros((len(c[:,]), num_clr))
    for j in range(num_clr):
        sincha_array=finallist[i+j]
        intensity=sincha_array[y,x]
        
        #signal value correction
        intecorrect = intensity/np.percentile(intensity, 99.99) 
        brightarray[:,j]=intecorrect
        round_num=int((i+num_clr)/num_clr)
        df[f"R{round_num}seq{j+1}"]=intecorrect
    
    decode_list=[]
    for k in range(len(c[:,])):
        brik=brightarray[k]
        
        if np.amax(brik)<0.15: 
            code=9
        elif np.amax(brik)/np.sum(brik)<0.5:
            code=9
        else:
            code=np.argmax(brik)+1
        
        decode_list.append(code)
        
    decode_array=np.array(decode_list)
        
    z=int(i/num_clr)
    decode[:,z]=decode_array
    
decode_result=[]
for m in range(len(c[:,])):
    list_d=decode[m,:]
    d=int("".join(map(str, list_d)))
    decode_result.append(d)
    
df["Decode"]=decode_result


#retrieve the number of barcode which passed quality check   
decode_str=[]
for i in range(len(decode_result)):
    s=str(decode_result[i])
    decode_str.append(s)

decode_qualified=[i for i in decode_str if "9" not in i]

            
        
   
                   
                   
        
    



In [57]:
#assign gene name to decoded color code
genelist=[]
decolist=df_genedecode.Decode.tolist()

for i in range(len(df)):
    a1=df.loc[i, "Decode"]
    a2=df_genedecode.loc[df_genedecode.Decode==a1, "Gene"]
    
    if a1 not in decolist:
        a3="NaN"
    else:
        a3=a2.values[0]
        
    genelist.append(a3)

df["Gene"]=genelist





In [58]:
df_genelist=df.Gene.tolist()
decode_qualified2=[i for i in df_genelist if "NaN" not in i]


In [59]:
# image: image in which gene spot is drawn
image=cv2.imread(f"{input_dir}\\{img_written}")


#to make spreadsheet showing the number of amplicaons for each gene
gnlist=[]
ampnum=[]

genenameset=set(df_genedecode["Gene"].tolist())
genenamelist=list(genenameset)


for i in range(len(genenamelist)):  #  3 can be len(df_genedecode.Gene)
    fig = plt.figure(figsize=(16, 4), dpi=300)
    
    genename=genenamelist[i]
    xlist=df.loc[df.Gene==genename, "x_pos"].tolist()
    ylist=df.loc[df.Gene==genename, "y_pos"].tolist()
    
    gnlist.append(genename)
    ampnum.append(len(xlist))
    
    plt.imshow(image, cmap="gray")
    plt.autoscale(False)
    plt.plot(xlist,ylist, 'ro', markersize=1)
    plt.savefig(f"{output_dir}\\image_results\\{genename}.png", bbox_inches = 'tight', dpi=300)
    
    
    
    plt.clf()
    plt.close()

df_summary = pd.DataFrame(columns=['Gene', 'amplicon_#'])

df_summary["Gene"]=gnlist
df_summary["amplicon_#"]=ampnum

df_summary.to_csv(f"{output_dir}\\num_amplicon_report.csv")
df.to_csv(f"{output_dir}\\all_data.csv")

decode_report={"all_decode" : len(df), "qualified1" : len(decode_qualified), "qualified2": len(decode_qualified2)}
df_decode_report = pd.DataFrame.from_dict(decode_report, orient='index').T
df_decode_report.to_csv(f"{output_dir}\\amplicon_quality.csv")


# code is ended here






In [60]:
# actb, hoxb9, hoxa3, hoxb3 and hoxd4 were seperately analyzed to avoid optical crowding in tisse1, 2, 3 of 96h and tissue4 of 120h
# actb and hoxb9 were seperately analyzed in tisse1, 3 of 120h

actb=cv2.imread(f"{input_dir}\\input_image\\{tissue_num}\\round5_c3.tif", cv2.IMREAD_GRAYSCALE)
hoxb9=cv2.imread(f"{input_dir}\\input_image\\{tissue_num}\\round5_c6.tif", cv2.IMREAD_GRAYSCALE)
hoxa3=cv2.imread(f"{input_dir}\\input_image\\{tissue_num}\\round5_c1.tif", cv2.IMREAD_GRAYSCALE)
hoxb3=cv2.imread(f"{input_dir}\\input_image\\{tissue_num}\\round5_c4.tif", cv2.IMREAD_GRAYSCALE)
hoxd4=cv2.imread(f"{input_dir}\\input_image\\{tissue_num}\\round5_c5.tif", cv2.IMREAD_GRAYSCALE)

kernel = np.ones((5, 5), np.uint8)

gene1=cv2.morphologyEx(actb, cv2.MORPH_TOPHAT, kernel)
gene2=cv2.morphologyEx(hoxb9, cv2.MORPH_TOPHAT, kernel)
gene3=cv2.morphologyEx(hoxa3, cv2.MORPH_TOPHAT, kernel)
gene4=cv2.morphologyEx(hoxb3, cv2.MORPH_TOPHAT, kernel)
gene5=cv2.morphologyEx(hoxd4, cv2.MORPH_TOPHAT, kernel)


c1=feature.blob_log(gene1, min_sigma=1, max_sigma=3, num_sigma=2,threshold=0.05, overlap=0.5, log_scale=False, exclude_border=False)
c2=feature.blob_log(gene2, min_sigma=1, max_sigma=3, num_sigma=2,threshold=0.05, overlap=0.5, log_scale=False, exclude_border=False)
c3=feature.blob_log(gene3, min_sigma=1, max_sigma=3, num_sigma=2,threshold=0.05, overlap=0.5, log_scale=False, exclude_border=False)
c4=feature.blob_log(gene4, min_sigma=1, max_sigma=3, num_sigma=2,threshold=0.05, overlap=0.5, log_scale=False, exclude_border=False)
c5=feature.blob_log(gene5, min_sigma=1, max_sigma=3, num_sigma=2,threshold=0.05, overlap=0.5, log_scale=False, exclude_border=False)


y_float=c2[:,0].tolist()
x_float=c2[:,1].tolist()

x=[]
y=[]
genename="HOXB9"
namelist = []
for i in range(len(c2[:])):
    a=int(x_float[i])
    x.append(a)
    b=int(y_float[i])
    y.append(b)
    
    namelist.append(genename)

del y_float
del x_float

d = {"x_pos" : x, "y_pos" : y, "Gene" : namelist}
df = pd.DataFrame.from_dict(d, orient = "index").T
df.to_csv(f"{output_dir}\\HOXB9_position.csv")

image=cv2.imread(f"{input_dir}\\{img_written}")


plt.imshow(image, cmap="gray")
plt.autoscale(False)
plt.plot(x,y, 'ro', markersize=1)
plt.savefig(f"{output_dir}\\image_results\\{genename}.png", bbox_inches = 'tight', dpi=300)
    
    
    
plt.clf()
plt.close()



In [61]:
y_float=c1[:,0].tolist()
x_float=c1[:,1].tolist()

x=[]
y=[]
genename="ACTB"
namelist = []
for i in range(len(c1[:])):
    a=int(x_float[i])
    x.append(a)
    b=int(y_float[i])
    y.append(b)
    
    namelist.append(genename)

del y_float
del x_float

d = {"x_pos" : x, "y_pos" : y, "Gene" : namelist}
df = pd.DataFrame.from_dict(d, orient = "index").T
df.to_csv(f"{output_dir}\\ACTB_position.csv")

image=cv2.imread(f"{input_dir}\\{img_written}")


plt.imshow(image, cmap="gray")
plt.autoscale(False)
plt.plot(x,y, 'ro', markersize=1)
plt.savefig(f"{output_dir}\\image_results\\{genename}.png", bbox_inches = 'tight', dpi=300)
    
    
    
plt.clf()
plt.close()

In [62]:
y_float=c3[:,0].tolist()
x_float=c3[:,1].tolist()

x=[]
y=[]
genename="HOXA3"
namelist = []
for i in range(len(c3[:])):
    a=int(x_float[i])
    x.append(a)
    b=int(y_float[i])
    y.append(b)
    
    namelist.append(genename)

del y_float
del x_float

d = {"x_pos" : x, "y_pos" : y, "Gene" : namelist}
df = pd.DataFrame.from_dict(d, orient = "index").T
df.to_csv(f"{output_dir}\\HOXA3_position.csv")

image=cv2.imread(f"{input_dir}\\{img_written}")


plt.imshow(image, cmap="gray")
plt.autoscale(False)
plt.plot(x,y, 'ro', markersize=1)
plt.savefig(f"{output_dir}\\image_results\\{genename}.png", bbox_inches = 'tight', dpi=300)
    
    
    
plt.clf()
plt.close()

In [63]:
y_float=c4[:,0].tolist()
x_float=c4[:,1].tolist()

x=[]
y=[]
genename="HOXB3"
namelist = []
for i in range(len(c4[:])):
    a=int(x_float[i])
    x.append(a)
    b=int(y_float[i])
    y.append(b)
    
    namelist.append(genename)

del y_float
del x_float

d = {"x_pos" : x, "y_pos" : y, "Gene" : namelist}
df = pd.DataFrame.from_dict(d, orient = "index").T
df.to_csv(f"{output_dir}\\HOXB3_position.csv")

image=cv2.imread(f"{input_dir}\\{img_written}")


plt.imshow(image, cmap="gray")
plt.autoscale(False)
plt.plot(x,y, 'ro', markersize=1)
plt.savefig(f"{output_dir}\\image_results\\{genename}.png", bbox_inches = 'tight', dpi=300)
    
    
    
plt.clf()
plt.close()


In [64]:
y_float=c5[:,0].tolist()
x_float=c5[:,1].tolist()

x=[]
y=[]
genename="HOXD4"
namelist = []
for i in range(len(c5[:])):
    a=int(x_float[i])
    x.append(a)
    b=int(y_float[i])
    y.append(b)
    
    namelist.append(genename)

del y_float
del x_float

d = {"x_pos" : x, "y_pos" : y, "Gene" : namelist}
df = pd.DataFrame.from_dict(d, orient = "index").T
df.to_csv(f"{output_dir}\\HOXD4_position.csv")

image=cv2.imread(f"{input_dir}\\{img_written}")


plt.imshow(image, cmap="gray")
plt.autoscale(False)
plt.plot(x,y, 'ro', markersize=1)
plt.savefig(f"{output_dir}\\image_results\\{genename}.png", bbox_inches = 'tight', dpi=300)
    
    
    
plt.clf()
plt.close()
